# 10. 그래프 이론

## 실전 문제

### 팀 결성 // RE(틀림)

#### 제한
* 풀이 시간 20분
* 시간 제한 2초
* 메모리 제한 128MB

##### 아이디어
* 서로소 집합 알고리즘의 find, union 기능을 함수로 만들어서 사용하자.

#### 시간 복잡도
* $O(N + {find 연산 개수}(1+\log_{2-{find 연산 개수}/N}N))$ 라고 한다. 정확한 계산은 조금 복잡할 듯 하다.
    * [참고할 만한 링크](https://hazel-developer.tistory.com/272)

#### 해설 본 후
* N, M의 범위가 모두 최대 100,000이므로 경로 압축을 이용한 서로소 집합 자료구조를 활용해야 함.
    * 경로 압축을 이용하지 않으면 시간 복잡도가 O(NM)이고, 이용하면 $O(N + {find 연산 개수}(1+\log_{2-{find 연산 개수}/N}N))$ 이므로 이 경우엔 경로 압축을 해야 시간 내에 해결 가능
* 기왕이면 가독성을 위해 함수를 앞에다 설정하고, 함수를 앞에다 설정하기 위해 팀을 기록할 리스트를 파라미터에 추가하자.
* 함수들을 잘못 짰다. find를 만들었으면 union에 이를 사용해야 하는데 그렇게 하지 않아 경로 압축이 안 됐을 것이다. 테스트 케이스에는 운좋게 통과가 되었다.
* 같은 팀 여부 확인 연산을 굳이 함수로 만들 필요 없이, find를 두 번 이용하여 바로 비교할 수 있다.

In [11]:
# 학생 번호 N, 연산 개수 M 입력
N, M = map(int, input().split())

# 학생들의 팀을 기록할 리스트 선언(자기 자신을 팀으로 초기화)
teams = [i for i in range(N+1)]

# 해당 학생의 팀을 찾는 함수 정의
def find_team(a):
    if teams[a] != a:
        teams[a] = find_team(teams[a])
    return teams[a]

# 팀 합치기 연산 정의
def union_team(a, b):
    team_a = teams[a]
    team_b = teams[b]
    
    if team_a < team_b:
        teams[team_b] = team_a
    else:
        teams[team_a] = team_b
    
# 같은 팀 여부 확인 연산 정의
def check_same_team(a, b):
    if teams[a] == teams[b]:
        return 'YES'
    else:
        return 'NO'

# 연산 조건 입력 받아서 진행
for _ in range(M):
    oper, a, b = map(int, input().split())
    
    # 합치기 연산
    if oper == 0:
        union_team(a, b)
    # 같은 팀 여부 확인 연산
    else:
        print(check_same_team(a, b))

ValueError: not enough values to unpack (expected 2, got 0)

#### 해설 코드 참조

In [ ]:
# 특정 원소가 속한 집합을 찾기
def find_team(teams, a):
    # 루트 노드가 아니라면, 루트 노드를 찾을 때까지 재귀적으로 호출
    if teams[a] != a:
        teams[a] = find_team(teams, teams[a])
    return teams[a]

# 두 원소가 속한 집합을 합치기
def union_team(teams, a, b):
    team_a = find_team(teams, a)
    team_b = find_team(teams, b)
    
    if team_a < team_b:
        teams[team_b] = team_a
    else:
        teams[team_a] = team_b

N, M = map(int, input().split())
teams = [i for i in range(N+1)] # 부모 테이블 초기화(부모를 자기 자신으로)
    
# 각 연산을 하나씩 확인
for _ in range(M):
    oper, a, b = map(int, input().split())
    # 합집합(union) 연산인 경우
    if oper == 0:
        union_team(teams, a, b)
    # 찾기(find) 연산인 경우
    elif oper == 1:
        if find_team(teams, a) == find_team(teams, b):
            print('YES')
        else:
            print('NO')

NO
NO
YES


### 도시 분할 계획
https://www.acmicpc.net/problem/1647

#### 제한
* 풀이 시간 40분
* 시간 제한 2초
* 메모리 제한 256MB

#### 아이디어
* 크루스칼 이용하여 최소 신장 트리 만든 다음, 가장 높은 비용의 간선 하나를 제거하면 될 것 같다.
* 시간 초과가 난다. 입력값이 최대 1,000,000개가 될 수 있으므로 sys.stdin.readline()을 이용해보자 => 됐다.

#### 시간 복잡도
* 크루스칼 알고리즘을 사용했고, 간선의 개수가 M개이므로 O(MlogM).
* M은 최대 1,000,000이므로 시간 내에 해결이 가능하다.

#### 해설 본 후
* 어차피 간선이 유지비 기준 오름차순으로 정렬되어 있으므로, 최대 유지비의 길을 저장할 때 max()를 쓸 필요 없이 해당 값으로 갱신만 해주면 된다.
* 답지에선 sys.stdin.readline()을 안 쓰고 그냥 input()으로 했는데, 백준에서 채점 돌려보면 input()으로 하면 시간초과가 난다.

In [ ]:
import sys

# 어떤 집이 속한 마을을 확인하는 함수 선언
def find_group(groups, a):
    if groups[a] != a:
        groups[a] = find_group(groups, groups[a])
    return groups[a]

# 두 집이 속한 마을을 같은 마을로 합치는 함수 선언
def union_group(groups, a, b):
    group_a = find_group(groups, a)
    group_b = find_group(groups, b)
    
    if group_a < group_b:
        groups[group_b] = group_a
    else:
        groups[group_a] = group_b

# 집 개수 N, 길 개수 M 입력
N, M = map(int, input().split())

# 집들이 속한 마을 정보를 담는 리스트 선언(자신이 속한 마을을 자신의 마을로 초기화)
groups = [i for i in range(N+1)]

# 길의 정보 입력
roads = []
for _ in range(M):
    A, B, C = map(int, sys.stdin.readline().split())
    # A번 집에서 B번 집으로 가는 경로의 유지비가 C임.
    roads.append((C, (A, B)))

# 길을 유지비 기준 오름차순으로 정렬
roads.sort()

# 최소 신장 트리를 통해 구한 길의 전체 유지비와 길 중 가장 유지비가 높은 길의 유지비를 초기화
result = 0
max_cost = 0

# 모든 길을 확인하며
for cost, road in roads:
    # 길로 이어진 두 집의 속한 마을이 다르다면
    if find_group(groups, road[0]) != find_group(groups, road[1]):
        # 속한 마을을 합쳐주고
        union_group(groups, road[0], road[1])
        # 전체 유지비에 해당 길의 유지비를 추가하고
        result += cost
        # 지금까지 확인한 길보다 유지비가 비싸다면 최대 유지비를 가진 길을 갱신
        max_cost = max(max_cost, cost) ## 어차피 오름차순 정렬되어 있으므로 max()를 쓰지 않고 갱신해도 된다.

# 전체 유지비에서 가장 유지비가 큰 길을 빼서 마을 두 개를 생성할 수 있다.
print(result - max_cost)

8


### 커리큘럼 // RE(해결 못 함)

#### 제한
* 풀이 시간 50분
* 시간 제한 2초
* 메모리 제한 128MB

#### 아이디어
* 선수 강의의 개수가 적은 것부터 확인한다.
* 어떤 강의의 선수 강의 중 indegree가 같은 레벨에 있는 강의가 여러 개라면 그 중에 가장 시간이 오래 걸리는 것만 시간에 추가한다.
* 어떤 강의의 시간을 체크할 때 선수 강의 목록에 선수 강의와 선수 강의의 선수 강의가 함께 들어있다면, 후자의 시간을 중복 체크하지 않도록 한다.

* 근데 여기선 상위 강의의 정보에 선수 강의가 들어가 있어서 거꾸로 봐야할 것 같은데... => 방향을 뒤집어서 기본 코드와 비슷하게 가보자

#### 시간 복잡도
* 해설 코드 기준 최악의 경우엔 O(N!)이 되지 않나? N번 강의가 1번부터 N-1번까지를 모두 선수 과목으로 한다면 간선이 그렇게 될 것 같은데
    * 아니구나. 노드 N개일 때, 간선은 N(N-1)/2이 최대가 되겠네. 그러면 O(N^3)이고 N은 최대 500이니까 해결 가능이겠다.

#### 해설 본 후
* 너무 어렵게 생각했나... 그리고 스스로가 답이라고 생각한 방식에 확신이 없고 구체화를 못해서 금방 포기해버린 건 아닌가 싶다.
* 상위 강의 -> 하위 강의의 형태로 입력이 주어지므로, 위상 정렬 알고리즘 사용을 위해 이를 뒤집어줘야 한다. 생각은 했는데, 다른 좋은 방법이 있지 않을까 하며 시도를 안 해봤다...
* 특히 같은 차수에서 둘 다 선수 과목으로 하는 과목이 있을 경우 더 많은 시간이 걸리는 것을 채택해야 하는데 그 부분을 어떻게 구현할지 막막했다. 해설에서는 똑같이 위상정렬을 돌되, max()를 이용하여 그 부분을 체크해줬다.
* 모르겠으면 그냥 많이 풀어보는 게 장땡인 건 맞을 것 같다. 정진합시다.

In [ ]:
# X

from collections import deque

# 듣고자 하는 강의 수 N 입력
N = int(input())


course_times = [0] * (N+1) # 각 강의의 소요 시간
graph = [[] for _ in range(N+1)]
indegrees = [0] * (N+1)

# 강의 시간 및 선수 강의 정보 입력
for i in range(1, N+1):
    time, *precourse = map(int, input().split())
    course_times[i] = time
    for p in precourse[:-1]:
        graph[p].append(i)
        indegrees[i] += 1

shortest_time = [0] * (N+1)

q = deque()

for i in range(1, N+1):
    if indegrees[i] == 0:
        q.append(i)
        
while q:
    course = q.popleft()
    

[0, 10, 10, 4, 4, 3]
[[], [], [1], [1], [3, 1], [3]]


#### 해설 코드 참고

In [ ]:
from collections import deque
import copy

# 노드 개수 입력
v = int(input())
# 모든 노드에 대한 진입차수를 0으로 초기화
indegree = [0] * (v+1)
# 각 노드에 연결된 간선 정보를 담기 위한 연결 리스트(그래프) 초기화
graph = [[] for i in range(v+1)]
# 각 강의 시간을 0으로 초기화
time = [0] * (v+1)

# 방향 그래프의 모든 간선 정보를 입력받기
for i in range(1, v+1):
    data = list(map(int, input().split()))
    time[i] = data[0] # 첫 번째 수는 시간 정보를 담고 있음
    for x in data[1:-1]:
        indegree[i] += 1
        graph[x].append(i) ## 여기도 키포인트
        
# 위상 정렬 함수
def topology_sort():
    result = copy.deepcopy(time) # 알고리즘 수행 결과를 담을 리스트
    q = deque() # 큐 기능을 위한 deque 라이브러리 사용
    
    # 처음 시작할 때는 진입차수가 0인 노드를 큐에 삽입
    for i in range(1, v + 1):
        if indegree[i] == 0:
            q.append(i)
            
    # 큐가 빌 때까지 반복
    while q:
        # 큐에서 원소 꺼내기
        now = q.popleft()
        # 해당 원소와 연결된 노드들의 진입차수에서 1 빼기
        for i in graph[now]:
            result[i] = max(result[i], result[now] + time[i]) ## 여기가 키포인트라고 생각한다.
            indegree[i] -= 1
            # 새롭게 진입차수가 0이 되는 노드를 큐에 삽입
            if indegree[i] == 0:
                q.append(i)
                
    # 위상 정렬 수행 결과 출력
    for i in range(1, v+1):
        print(result[i])

topology_sort()

## Q41. 여행 계획

#### 제한
* 풀이 시간 40분
* 시간 제한 1초
* 메모리 제한 128MB

#### 아이디어
* 여행 계획이 가능하려면 여행 계획 내의 모든 여행지가 한 그룹에 있어야 한다 => 루트 노드가 같아야 한다.
* 여행지 연결 여부가 인접 행렬로 들어오니 이를 인접 리스트의 형태로 바꿔주고, 인접 행렬을 받아올 때 중복이 생기므로 전체 리스트 내 원소는 set으로 구성하여 중복을 없앤다.
* 여행 계획 내 도시들이 모두 연결되어 있어야(루트 노드가 모두 같아야) 여행 계획이 가능하므로 첫 번째 노드의 루트 노드를 기록하여 다른 노드들의 루트 노드와 비교하여 루트 노드가 서로 다른 경우 NO, 모두 같은 경우는 YES를 출력한다

#### 시간 복잡도
* 경로 압축 이용한 서로소 집합 알고리즘의 시간 복잡도는 $O(N + {find 연산 개수}(1+\log_{2-{find 연산 개수}/N}N))$라고 한다.
* N은 최대 500이고, find 연산 개수도 최대 4N 정도일 것이므로 시간 내에 해결이 가능하다.

#### 해설 본 후
* 잘 풀었다.
* 답지에서는 여행 계획 내 여행지의 모든 루트 노드가 동일한지 체크할 때 확인하는 노드와 그 다음 노드의 루트 노드가 같은지 비교를 반복하며 체크했다.
    * 또한 답지에서는 비교할 때도 루트 노드를 찾을 때 find 함수를 썼는데, 어차피 union과정에서 다 갱신되어 그냥 루트 노드 리스트에서 뽑아오는 것과 같겠지만, 이게 더 안전할 것 같긴 하다.

* 굳이 플래그 쓸 필요 없이 바로 멈추면 된다.

In [ ]:
# 루트 노드 찾는 함수 선언
def find_root(roots, node):
    if roots[node] != node:
        roots[node] = find_root(roots, roots[node])
    return roots[node]

# 두 노드의 루트 노드를 같게하여 같은 묶음으로 묶는 함수 선언
def union_root(roots, a, b):
    root_a = find_root(roots, a)
    root_b = find_root(roots, b)
    
    if root_a < root_b:
        roots[root_b] = root_a
    else:
        roots[root_a] = root_b

# 여행지 수 N, 여행 계획에 속한 도시 수 M 입력
N, M = map(int, input().split())

# 각 여행지의 루트 노드는 자기 자신으로 초기화
roots = [i for i in range(N+1)]

# 여행지 연결 정보 입력
for i in range(1, N+1):
    row = list(map(int, input().split()))
    for j in range(N):
        # 연결돼있는 도시들은 그때그때 union을 통해 루트노드를 같게 하여 한 그룹으로 만들어주기
        if row[j] == 1:
            union_root(roots, i, j+1)

# 여행 계획 정보 입력
travel_plan = list(map(int, input().split()))

# 여행 계획의 첫 번째 도시의 루트 노드를 할당
travle_root = roots[travel_plan[0]]
# 플래그 변수 선언
flag = True

# 두 번째 도시부터 확인하며
for node in travel_plan[1:]:
    # 첫 번째 도시의 루트 노드와 확인하는 도시의 루트 노드가 다른 경우
    if roots[node] != travle_root:
        # 플래그 변수를 False로 바꾸고 반복 중지
        flag = False
        break
# 반복이 종료됐을 때 플래그 변수가 True라면 모든 도시의 루트 노드가 같아 연결되어 있다는 뜻이므로 YES 출력, 아니라면 NO 출력
else:
    if flag == True: ## 그냥 if flag: 만 해도 됨
        print('YES')
    else:
        print('NO')

YES


#### 해설 코드 참고(루트 노드 비교 부분)

In [ ]:
for i in range(M-1):
    if find_root(roots, travel_plan[i]) != find_root(roots, travel_plan(i+1)):
        flag = False

# Q42. 탑승구 // RE(해결 못 함)

#### 제한
* 풀이 시간 50분
* 시간 제한 1초
* 메모리 제한 128MB

#### 아이디어
* 비행기가 들어갈 수 있는 가장 높은 번호의 탑승구에부터 배치 -> 그 탑승구가 이미 차 있는 경우 탑승구 번호를 하나씩 감소시키면서 빈 곳에 배치
* (기억의 저편) 탑승구 확인 때마다 번호 하나 작은 탑승구랑 union해서 푸는 거였는데.. => 맞다.

#### 시간 복잡도
* $O(N + {find 연산 개수}(1+\log_{2-{find 연산 개수}/N}N))$ ? 
* 해설 코드 기준 대충 union연산과 find연산을 각각 최대 100,000번씩 하고, 그러면 시간 내에 해결 가능할 듯?

#### 해설 본 후
* 거의 다 왔는데 답을 봐버렸다. 생각한 것들을 다 해봤으면 맞았을 텐데. 포기가 빨랐다.
* 마지막에 union해줄 때 해당 노드가 아닌 부모 노드로 해줘야 답이 된다.

In [ ]:
# O(G*P)이고, G, P가 각각 최대 100,000이므로 제한 시간 초과 가능성 농후

# 탑승구 수 G 입력
G = int(input())

# 비행기 수 P 입력
P = int(input())

# 탑승구가 찼는지 확인하는 리스트 선언
full = [False] * (G+1)

# 정답 변수와 플래그 변수 선언
answer = 0
flag = True

# 비행기별 도킹 가능 탑승구 정보 받아오며
for _ in range(P):
    biggest_gate = int(input())
    # 플래그 변수가 True라면 탑승구 갱신 진행
    if flag:
        # 탑승구 번호 역순으로 확인하며
        for i in range(biggest_gate, -1, -1):
            # 모든 탑승구를 확인했는데 비행기를 도킹할 수 없다면 플래그 변수를 False로 변환
            if i == 0:
                flag = False
                continue
            # 해당 탑승구에 비행기를 도킹할 수 있다면 탑승구 참 여부를 갱신하고 정답 변수에 1 추가 후 다음 비행기 체크
            if not full[i]:
                full[i] = True
                answer += 1
                break

print(answer)

2


In [ ]:
# 해당 노드의 루트 노드 찾기(특정 원소가 속한 집합 찾기)
def find_parent(parents, node):
    if parents[node] != node:
        parents[node] = find_parent(parents, parents[node])
    return parents[node]

# 두 원소의 루트 노드 통일(두 원소가 속한 집합 합치기)
def union_parent(parents, a, b):
    parent_a = find_parent(parents, a)
    parent_b = find_parent(parents, b)
    
    if parent_a < parent_b:
        parents[parent_b] = parent_a
    else:
        parents[parent_a] = parent_b

# 탑승구 수 G 입력
G = int(input())

# 비행기 수 P 입력
P = int(input())

parents = [i for i in range(G+1)]

### my version
# 비행기별 도킹 가능 탑승구 정보 입력
boarding = [0]
for _ in range(P):
    boarding.append(int(input()))

# 정답 변수 선언
answer = 0

# 비행기의 탑승구를 확인하면서
for i in range(1, P+1):
    # 루트 노드가 0이면 반복 종료
    if find_parent(parents, boarding[i]) == 0:
        break
    # 아니라면 정답 변수에 1 추가하고 해당 탑승구의 루트 노드와 그 왼쪽 노드의 그룹을 합침
    answer += 1
    union_parent(parents, parents[boarding[i]]-1, parents[boarding[i]]) ## 이 부분을 부모 노드가 아닌 기본 노드로 했었음
###

### 해설 버전
answer = 0
for _ in range(P):
    data = find_parent(parents, int(input())) # 현재 비행기의 탑승구의 루트 확인
    if data == 0: # 현재 루트가 0이라면, 종료
        break
    union_parent(parents, data, data - 1) # 그렇지 않다면 바로 왼쪽의 집합과 합치기
    answer += 1
###

print(answer)

2


## Q43. 어두운 길

 #### 제한
 * 풀이 시간 40분
 * 시간 제한 1초
 * 메모리 제한 128MB

#### 아이디어
* 크루스칼 써서 최소신장트리 구한 후 전체 비용에서 최소신장트리 구성 비용 빼기

#### 시간 복잡도
* 크루스칼 알고리즘의 시간 복잡도는 O(ElogE).
* 이 문제에서 E는 M과 같고, M의 최댓값은 200,000이므로 시간 내에 해결이 가능하다.

#### 해설 본 후
* 잘 풀었다

In [ ]:
# 노드가 속한 집합 찾기
def find_root(roots, node):
    if roots[node] != node:
        roots[node] = find_root(roots, roots[node])
    return roots[node]

# 두 노드의 집합 합치기
def union_root(roots, a, b):
    root_a = find_root(roots, a)
    root_b = find_root(roots, b)
    
    if root_a < root_b:
        roots[root_b] = root_a
    else:
        roots[root_a] = root_b

# 집의 수 N과 도로의 수 M 입력
N, M = map(int, input().split())

# 도로 정보 및 전체 가로등 비용 입력
graph = []
whole_cost = 0

for _ in range(M):
    X, Y, Z = map(int, input().split())
    graph.append((Z, X, Y))
    whole_cost += Z

# 도로를 비용에 따라 오름차순으로 정렬
graph.sort()

# 루트 노드 테이블 선언
roots = [i for i in range(N)]

# 최소 신장 트리의 비용값 선언
total = 0
# 비용이 적은 순으로 모든 간선을 확인하며
for cost, x, y in graph:
    # 간선으로 연결된 두 노드가 같은 그룹이 아닌 경우(사이클이 발생하지 않는 경우) 두 집합을 합치고 비용값에 비용 추가
    if find_root(roots, x) != find_root(roots, y):
        union_root(roots, x, y)
        total += cost

# 정답 출력
print(whole_cost-total)

51


## Q44. 행성 터널 // RE(해결 못 함)
https://www.acmicpc.net/problem/2887

#### 제한
* 풀이 시간 40분
* 시간 제한 1초
* 메모리 제한 128MB

#### 아이디어
* (기억의 저편) 축 별로 정렬해서 전체 반복횟수를 줄여야 했던 것 같은데
* 일단 모든 간선을 전부 고려하여 코드를 짜보자
* x, y, z가 최소가 되는 그룹을 각각 나누면? => 결국 마지막에 합쳐서 계산해야 해서 똑같은데...

#### 시간 복잡도
* 공간 복잡도 => 질문에서 발췌
    * n 이 100000 일 때 v 안에 들어가는 원소의 개수는 100000 * 99999 / 2 = 4999950000 개 입니다. 각 원소는 int 3개 = 12 byte 이므로, 필요한 메모리의 양은 4999950000 * 12 = 약 56 TB 으로, 문제에서 주어진 제한 128 MB 를 훨씬 초과합니다.
* 시간복잡도도, 전체 간선으로 모두 고려하게 되면 O(N^2*logN^2)이 되어버려 시간 내에 해결할 수 없다.
* 축 별로 N-1개의 간선만 고려하게 되면 O(NlogN)의 시간복잡도를 가질 것이므로 시간 내에 해결이 가능하다.

#### 해설 본 후
* 한 축을 기준으로 정렬했을 때 인접한 노드끼리만을 간선으로 잇는다면 N-1개의 간선만 고려하여 최소 신장 트리를 만들 수 있다. 이를 x, y, z 축에 모두 적용하면 N(N-1)/2개가 아닌 3(N-1)개의 간선만 이용하여 최소 신장 트리를 만들 수 있다.
    * 이렇게 정렬할 거라면 x, y, z 리스트에 모든 좌표가 다 들어갈 필요 없이 해당하는 축의 좌표만 들어가면 된다.
* 답지 코드도 시간 초과가 난다(...)
    * sys.stdin.readline() 사용하거나 pypy3로 돌리면 통과

In [ ]:
## 1트
## 전체 간선을 고려한 코드 => 메모리 초과

# 두 노드 간 비용 계산하는 코드
def cal_cost(a, b):
    return min(abs(a[0] - b[0]), abs(a[1] - b[1]), abs(a[2] - b[2]))

# 루트 노드 찾고 갱신하는 노드
def find_root(roots, node):
    if roots[node] != node:
        roots[node] = find_root(roots, roots[node])
    return roots[node]

# 두 노드의 그룹을 합치는 코드
def union_root(roots, a, b):
    root_a = find_root(roots, a)
    root_b = find_root(roots, b)
    
    if root_a < root_b:
        roots[root_b] = root_a
    else:
        roots[root_a] = root_b

# 행성 개수 N 입력
N = int(input())

# 각 행성의 좌표 입력
planets = []
for _ in range(N):
    planets.append(tuple(map(int, input().split())))

# 루트 노드 리스트 초기화
roots = [i for i in range(N)]

# 가능한 모든 간선을 담는 리스트 선언
tunnels = []
for i in range(N):
    for j in range(i+1, N):
        tunnels.append((cal_cost(planets[i], planets[j]), i, j))

# 간선을 비용 기준 오름차순 정렬
tunnels.sort()

# 크루스칼 알고리즘 시행
total = 0
for cost, a, b in tunnels:
    if find_root(roots, a) != find_root(roots, b):
        union_root(roots, a, b)
        total += cost

print(total)

4


In [ ]:
## 2트
## 기억을 되살려보자 => 사실상 위랑 똑같은 코드... 오히려 비효율적일지도

# 두 노드 간 비용 계산하는 코드
def cal_cost(a, b):
    return min(abs(a[0] - b[0]), abs(a[1] - b[1]), abs(a[2] - b[2]))

# 루트 노드 찾고 갱신하는 노드
def find_root(roots, node):
    if roots[node] != node:
        roots[node] = find_root(roots, roots[node])
    return roots[node]

# 두 노드의 그룹을 합치는 코드
def union_root(roots, a, b):
    root_a = find_root(roots, a)
    root_b = find_root(roots, b)
    
    if root_a < root_b:
        roots[root_b] = root_a
    else:
        roots[root_a] = root_b

# 행성 개수 N 입력
N = int(input())

# 각 행성의 좌표 입력
planets = []
for _ in range(N):
    planets.append(tuple(map(int, input().split())))

# 루트 노드 리스트 초기화
roots = [i for i in range(N)]

# 비용 체크할 때 선택된 축을 기준으로 나눠서 각각의 리스트에 저장 
xs = []
ys = [] 
zs = []
for i in range(N-1):
    for j in range(i+1, N):
        x_cost = abs(planets[i][0] - planets[j][0])
        y_cost = abs(planets[i][1] - planets[j][1])
        z_cost = abs(planets[i][2] - planets[j][2])
        if cal_cost(planets[i], planets[j]) == x_cost:
            xs.append((x_cost, i, j))
        elif cal_cost(planets[i], planets[j]) == y_cost:
            ys.append((y_cost, i, j))
        else:
            zs.append((z_cost, i, j))

# 해당 간선을 모두 합쳐 정렬
tunnels = xs + ys + zs
tunnels.sort()

# 크루스칼 시행
total = 0
for cost, a, b in tunnels:
    if find_root(roots, a) != find_root(roots, b):
        union_root(roots, a, b)
        total += cost

print(total)

4


In [ ]:
## 3트: 시간 초과 // sys 쓰면 통과

# 두 노드 간 비용 계산하는 코드
def cal_cost(a, b):
    return min(abs(a[0] - b[0]), abs(a[1] - b[1]), abs(a[2] - b[2]))

# 루트 노드 찾고 갱신하는 노드
def find_root(roots, node):
    if roots[node] != node:
        roots[node] = find_root(roots, roots[node])
    return roots[node]

# 두 노드의 그룹을 합치는 코드
def union_root(roots, a, b):
    root_a = find_root(roots, a)
    root_b = find_root(roots, b)
    
    if root_a < root_b:
        roots[root_b] = root_a
    else:
        roots[root_a] = root_b

# 행성 개수 N 입력
N = int(input())

# 각 행성의 좌표 입력 및 루트 노드 리스트 초기화
planets = []
roots = []
for num in range(N):
    planets.append(list(map(int, input().split())) + [num])
    roots.append(num)

# 각 축 기준으로 오름차순 정렬한 리스트를 각각 선언
xs = sorted(planets, key=lambda x:x[0])
ys = sorted(planets, key=lambda x:x[1])
zs = sorted(planets, key=lambda x:x[2])

# 위 리스트에 대해 최소신장트리를 만드는 간선들만을 간선 후보에 추가
tunnels = []
for i in range(N-1):
    tunnels.append((cal_cost(xs[i], xs[i+1]), xs[i][3], xs[i+1][3]))
    tunnels.append((cal_cost(ys[i], ys[i+1]), ys[i][3], ys[i+1][3]))
    tunnels.append((cal_cost(zs[i], zs[i+1]), zs[i][3], zs[i+1][3]))

# 간선 후보가 담긴 리스트를 비용 기준 오름차순으로 정렬
tunnels.sort()

# 크루스칼 시행
total = 0
for cost, a, b in tunnels:
    if find_root(roots, a) != find_root(roots, b):
        union_root(roots, a, b)
        total += cost

print(total)

4


#### 해설 코드 참조

In [ ]:
# 특정 원소가 속한 집합을 찾기
def find_parent(parent, x):
    # 루트 노드가 아니라면, 루트 노드를 찾을 때까지 재귀적으로 호출
    if parent[x] != x:
        parent[x] = find_parent(parent, parent[x])
    return parent[x]

# 두 원소가 속한 집합을 합치기
def union_parent(parent, a, b):
    a = find_parent(parent, a)
    b = find_parent(parent, b)
    if a < b:
        parent[b] = a
    else:
        parent[a] = b
        
# 노드의 개수 입력받기
n = int(input())
parent = [0] * (n+1) # 부모 테이블 초기화

# 모든 간선을 담을 리스트와 최종 비용을 담을 ㅂ녀수
edges = []
result = 0

# 부모 테이블상에서, 부모를 자기 자신으로 초기화
for i in range(1, n+1):
    parent[i] = i
    
x = []
y = []
z = []

# 모든 노드에 대한 좌표 값 입력받기
for i in range(1, n+1):
    data = list(map(int, input().split()))
    x.append((data[0], i))
    y.append((data[1], i))
    z.append((data[2], i))
    
x.sort()
y.sort()
z.sort()

# 인접한 노드들로부터 간선 정보를 추출하여 처리
for i in range(n-1):
    # 비용순으로 정렬하기 위해서 튜플의 첫 번재 원소를 비용으로 설정
    edges.append((x[i+1][0] - x[i][0], x[i][1], x[i+1][1]))
    edges.append((y[i+1][0] - y[i][0], y[i][1], y[i+1][1]))
    edges.append((z[i+1][0] - z[i][0], z[i][1], z[i+1][1]))

# 간선을 비용순으로 정렬
edges.sort()


# 간선을 하나씩 확인하며
for edge in edges:
    cost, a, b = edge
    # 사이클이 발생하지 않는 경우에만 집합에 포함
    if find_parent(parent, a) != find_parent(parent, b):
        union_parent(parent, a, b)
        result += cost

print(result)

## Q45. 최종 순위 // RE(해결 못 함)
https://www.acmicpc.net/problem/3665

#### 제한
* 풀이 시간 60분
* 시간 제한 1초
* 메모리 제한 256MB

#### 아이디어
* 진입차수를 통한 위상정렬로 풀어야 할 것 같다. 진입 차수를 이용해 상대적 등수가 바뀐 경우 대상 팀의 진입차수에 각각 +1 -1을 해주고, 이 과정을 모두 진행한 후 진입차수가 모두 유니크하다면 등수 매기기 가능, 애매한 것들은 ?로 처리하고, 아예 잘못됐다면 impossible로 하면 될듯?
    * 이것도 아니다. 진입차수만 바꿔서 적용하면 다른 팀과의 등수 변화가 생기는 경우가 있다. => 다른 팀과의 상대적 등수를 보존하지 못하는 경우가 생긴다.
* 등수를 진입차수의 개념으로 바로 사용하면 효율적일 것 같다. 상대적인 등수가 바뀌었을 때, 등수가 더 높아진(작은 숫자가 된)다면 등수에 -1, 반대라면 +1을 하는 식으로.
    * 아니다. 어차피 이전 등수 기준으로 상대적인 등수가 결정되므로 이를 계속 사용해야 한다. 즉, 이전 등수를 직접 수정하는 것은 안 되기 때문에 어차피 새로운 변수를 하나 만들어야 한다.

#### 시간 복잡도
* 정답 기준, 위상 정렬을 이용하므로 O(V+E)겠고, 이 경우에선 O(N^2)가 될 것이다. N은 최대 500이므로 시간 내에 해결이 가능하다.

#### 해설 본 후
* 애초에 문제 이해도 잘못했다.
* 진입 차수만 생각할 것이 아니라, 그래프를 만들어 위상정렬 역시 적용해야 한다. 내 풀이 코드엔 위상정렬이 적용되지 않았다.
* 일관성이 없는 경우 => 사이클 발생한 경우 라는데... 솔직히 등수에 일관성이 없다는 게 무슨 말인지 이해를 못했다.

In [34]:
# 테스트 케이스 개수만큼 반복
for tc in range(int(input())):
    # 팀 개수 n 입력
    n = int(input())
    
    # 작년 등수 정보 입력
    grade = list(map(int, input().split()))
    
    # 새로운 등수 정보 선언
    new_grade = grade[:]
    
    # 상대적 등수가 바뀐 쌍의 수 m 입력
    m = int(input())
    
    # 등수 변화 쌍 입력 받아 등수 변화 적용
    for _ in range(m):
        a, b = map(int, input().split())
        if grade[a-1] > grade[b-1]:
            new_grade[a-1] -= 1
            new_grade[b-1] += 1
        else:
            new_grade[a-1] += 1
            new_grade[b-1] -= 1
    
    # 등수 처리를 원활히 해주기 위해 문자열로 변경
    str_grade = ''.join([str(i) for i in new_grade])
    
    # 등수가 겹치는 경우 해당 숫자를 ?로 변경
    for i in range(n):
        if str_grade.count(str_grade[i]) > 1:
            str_grade = str_grade.replace(str_grade[i], '?')
    
    # 전부 ?라면 일관성이 없으므로 IMPOSSIBLE 출력   
    if str_grade.count('?') == n:
        print('IMPOSSIBLE')
    # 아니라면 등수 출력(모르는 경우의 등수는 ?로 출력)
    else:
        for i in str_grade:
            print(i, end=' ')
    print()

2 3 1 


#### 해설 코드 참고

In [ ]:
from collections import deque

# 테스트 케이스만큼 반복:
for tc in range(int(input())):
    # 노드의 개수 입력 받기
    n = int(input())
    # 모든 노드에 대한 진입차수는 0으로 초기화
    indegree = [0] * (n+1)
    # 각 노드에 연결된 간선 정보를 담기 위한 인접 행렬 초기화
    graph = [[False] * (n+1) for i in range(n+1)]
    # 작년 순위 정보 입력
    data = list(map(int, input().split()))
    # 방향 그래프의 간선 정보 초기화
    for i in range(n):
        for j in range(i+1, n):
            graph[data[i]][data[j]] = True
            indegree[data[j]] += 1
            
    # 올해 변경된 순위 정보 입력
    m = int(input())
    for i in range(m):
        a, b = map(int, input().split())
        # 간선의 방향 뒤집기
        if graph[a][b]:
            graph[a][b] = False
            graph[b][a] = True
            indegree[a] += 1
            indegree[b] -= 1
        else:
            graph[a][b] = True
            graph[b][a] = False
            indegree[a] -= 1
            indegree[b] += 1
            
    # 위상 정렬 시작
    result = [] # 알고리즘 수행 결과를 담을 리스트
    q = deque() # 큐 기능을 위한 deque 라이브러리 사용
    
    # 처음 시작할 대는 진입차수가 0인 노드를 큐에 삽입
    for i in range(1, n+1):
        if indegree[i] == 0:
            q.append(i)
            
    certain = True # 위상 정렬 결과가 오직 하나인지의 여부
    cycle = True # 그래프 내 사이클이 존재하는지 여부
    
    # 정확히 노드의 개수만큼 반복
    for i in range(n):
        # 큐가 비어 있다면 사이클이 발생했다는 의미
        if len(q) == 0:
            cycle = True
            break
        # 큐의 원소가 2개 이상이라면 가능한 정렬 결과가 여러 개라는 ㄴ의미
        if len(q) >= 2:
            certain = False
            break
        # 큐에서 원소 꺼내기
        now = q.popleft()
        result.append(now)
        # 해당 원소와 연결된 노드들의 진입차수에서 1 빼기
        for j in range(1, n+1):
            if graph[now][j]:
                indegree[j] -= 1
                # 새롭게 진입차수가 0이 되는 노드를 큐에 삽입
                if indegree[j] == 0:
                    q.append(j)
    
    # 사이클이 발생하는 경우(일관성이 없는 경우)
    if cycle:
        print("IMPOSSIBLE")
    # 위상 정렬 결과가 여러 개인 경우
    elif not certain:
        print("?")
    else:
        for i in result:
            print(i, end=' ')
        print()
        